# Binairo 

Binairo is played on a rectangular grid with no standard size. Some cells start out filled with black or white circles. The rest of the cells are empty. The goal is to place circles in all cells in such a way that:

1. Each row and each column must contain an equal number of white and black circles.

2. More than two circles of the same color can't be adjacent.

3. Each row and column is unique. 

[Play On Janko](https://www.janko.at/Raetsel/Tohu-Wa-Vohu/index.htm)

---------

1. Binairo 在矩形网格上播放，没有标准尺寸。有些单元格开始时充满了黑色或白色的圆圈。其余的单元格是空的。目标是以以下方式在所有单元格中放置圆圈；
2. 每行和每列必须包含相同数量的白色和黑色圆圈。
3. 两个以上相同颜色的圆圈不能相邻。
4. 每一行每一列都是独一无二的。

![](https://www.janko.at/Raetsel/Tohu-Wa-Vohu/Regeln-01.gif) ![](https://www.janko.at/Raetsel/Tohu-Wa-Vohu/Regeln-02.gif)

In [1]:
def readGrid(path):
    with open(f"../assets/data/Binairo/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("131_8x8")
    print(m, n)
    for g in grid:
        print(g)

8 8
['-', '2', '2', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '1', '-', '-', '2']
['-', '-', '1', '-', '1', '1', '-', '-']
['-', '-', '-', '-', '-', '1', '1', '-']
['-', '-', '1', '-', '-', '-', '-', '-']
['-', '2', '-', '-', '-', '-', '-', '1']
['-', '-', '-', '1', '2', '1', '-', '2']
['1', '-', '-', '-', '1', '1', '-', '-']


In [2]:
from ortools.sat.python import cp_model as cp
class BinarioSolver:
    
    def __init__(self, X, Y, grid) -> None:
        self.X = X
        self.Y = Y
        self.grid = grid 
        self.x = {}
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        for i in range(self.X):
            for j in range(self.Y):
                if self.grid[i][j] == "-":
                    self.x[i, j] = self.model.NewBoolVar(f'x[{i}, {j}]')
                elif self.grid[i][j] == "1":
                    self.x[i, j] = 0
                else:
                    self.x[i, j] = 1
    def addConstr(self):
        for i in range(self.X):
            for j in range(self.Y - 2):
                arr = [self.x[i, y] for y in range(j, j + 3)]
                self.model.Add(sum(arr) <= 2)
                self.model.Add(sum(arr) >= 1)

        for j in range(self.Y):
            for i in range(self.X - 2):
                arr = [self.x[xx, j] for xx in range(i, i + 3)]
                self.model.Add(sum(arr) <= 2)
                self.model.Add(sum(arr) >= 1)
        
        for i in range(self.X):
            self.model.Add(sum(self.x[i,j] for j in range(self.Y)) == int(self.Y) // 2)
        for j in range(self.Y):
            self.model.Add(sum(self.x[i,j] for i in range(self.X)) == int(self.X) // 2)
    
    def printgrid(self):
        for i in range(self.X):
            for j in range(self.Y):
                if self.grid[i][j] == "2":
                    print("2", end = " ")
                elif self.grid[i][j] == "1":
                    print("1", end = " ")
                else:
                    if self.solver.Value(self.x[i, j]) > 1e-1:
                        print("2", end=" ")
                    else:
                        print("1", end=" ")
            print()
        print()
    
    def solve(self):
        self.solver.Solve(self.model)
        self.printgrid()
        # print(self.solver.ResponseStats())

if __name__ == "__main__":
    x, y, grid = readGrid("131_8x8")
    BinarioSolverTest = BinarioSolver(x, y, grid)
    BinarioSolverTest.addConstr()
    BinarioSolverTest.solve()
                

1 2 2 1 2 2 1 1 
2 1 1 2 1 2 1 2 
2 2 1 2 1 1 2 1 
1 2 2 1 2 1 1 2 
2 1 1 2 1 2 2 1 
2 2 1 1 2 2 1 1 
1 1 2 1 2 1 2 2 
1 1 2 2 1 1 2 2 

